# Snoqualmie Observations

In [1]:
# netcdf/numpy/xray/stats
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import xarray as xr
from scipy.stats.stats import pearsonr

# OS interaction
import sys, pickle, os

# import plotting
import seaborn as sns
import matplotlib
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap

# Offline Turbulence Package
import turbpy

# Customize
sns.set_style("whitegrid")
sns.set_context('paper')
%matplotlib inline

In [2]:
# --------------------------------------------------------------------------------------------------------------------
# Directory Lists
# Unix
if 'linux' in sys.platform:
    dirPre = '/home/lapok/gdrive/'
# Mac
elif 'darwin' in sys.platform:
    dirPre = '/Users/karllapo/gdrive/'

dirProj = dirPre + 'SnowHydrology/proj/ModTsfc/'
dirPrint = dirProj + '/Graphics'
dirData = dirProj + 'data'

# Original Snoqualmie observations
dirDataRaw = dirPre + 'GroundObs/Snoqualmie/Raw/SNQ_Data/'

# Sub-functions for calculating dewpoint/RH

In [3]:
def calc_RH(p,q,T):
    RH = np.empty_like(T)
    if np.nanmin(T) < 200:
        T = T + 273.16
    T_0 = 273.16

    RH = .263*p*q* (np.exp( (17.67*(T-T_0)) / (T-29.65) ))**(-1)
    RH[RH > 100] = 100
    return(RH)

def calc_Tdew(T,RH):
    # Unit checks
    if np.nanmax(T) > 100 or np.nanmin(T) > 40:
        raise ValueError('Air temperature must be in Celsius')

    if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
        RH = RH/100
        if np.nanmax(RH) > 1 or np.nanmin(RH) < 0:
            raise ValueError('Relative humidity must be a fraction on [0,1]')

    if not np.size(RH) == np.size(T):
        raise ValueError('Relative humidity and air temperature must have the same number of elements')

    # -----------------------------------------------------------------------------------------------
    # ALGORITHM
    # When are we calculating with respect to frost or water?
    frost_ind = np.flatnonzero(T <= 0)
    water_ind = np.flatnonzero(T > 0)
    # Frost coefficients 
    b_frost = 22.587
    c_frost = 273.86
    # Water coefficients
    b_water = 17.625
    c_water = 243.03

    # Pre-allocate
    Tdew = np.empty_like(T)

    Tdew[frost_ind] = MagnusTetens(T[frost_ind],RH[frost_ind],b_frost,c_frost)
    Tdew[water_ind] = MagnusTetens(T[water_ind],RH[water_ind],b_water,c_water)

    return(Tdew)

# SUB-FUNCTION for actual expression
def MagnusTetens(T,RH,b,c):
    dew = (c*( np.log(RH) + (b * T)/(c + T) )) / ( b - np.log(RH) - (b * T)/(c + T) )
    return(dew)

## Snoqualmie Model Forcing Data

In [4]:
# date parsing function for pandas csv_read
def parse(y, mo, dy, hr, mn):
    s = '00'
    date_str = y+':'+mo+':'+dy+'_'+hr+':'+mn+':'+s
    dt = datetime.strptime(date_str,"%Y:%m:%d_%H:%M:%S")
    return dt

# -------------------------------------------------------------------------
# Read forcing data
os.chdir(dirDataRaw  + 'Filled_Forcing')
datafile = 'Met_Forcing_SNQ_10_1_2012_5_11_2015.txt'
SNQ = pd.read_csv(datafile,
                  sep=r"\s*",
                  header=None,
                  parse_dates={'Datetime' : [0, 1, 2, 3, 4]},
                  date_parser=parse,
                  index_col='Datetime',
                  engine='python',)
# Format
SNQ.columns = ['seconds', 'precip', 'SWdwn', 'LWdwn', 'Tair', 'WIND', 'Press', 'QS']
SNQ.drop('seconds', axis=1, inplace=True)

# [K] -> [C]
SNQ.Tair = SNQ.Tair - 273.15

# -------------------------------------------------------------------------
# RH
SNQ['RH'] = calc_RH(SNQ.Press.values, SNQ.QS.values, SNQ.Tair.values)
# Dew point temperature
SNQ['Tdew'] = calc_Tdew(SNQ.Tair.values - 273.16, SNQ.RH.values)

# -------------------------------------------------------------------------
# Convert to xarray Dataset
SNQ = xr.Dataset.from_dataframe(SNQ)
SNQ = SNQ.rename({'Datetime': 'time'})


//anaconda/lib/python3.5/site-packages/pandas/io/parsers.py:1961: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
//anaconda/lib/python3.5/site-packages/pandas/io/parsers.py:1963: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:46: RuntimeWarning: divide by zero encountered in true_divide
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:46: RuntimeWarning: invalid value encountered in true_divide


## Read QC'ed half-hourly data

In [5]:
# date parsing function for pandas csv_read
def parse(y, mo, dy, hr, mn):
    s = '00'
    date_str = y+':'+mo+':'+dy+'_'+hr+':'+mn+':'+s
    dt = datetime.strptime(date_str,"%Y:%m:%d_%H:%M:%S")
    return dt

# -------------------------------------------------------------------------
# Read supporting met data
os.chdir(dirDataRaw + 'Quality_Controled')
datafile = 'SNQ_QC_30min_2013_2015.csv'
support = pd.read_csv(datafile,
                      sep=',',
                      header=0,
                      na_values=-9999,
                      skiprows=[1,2],
                      parse_dates={'Datetime' : [0, 1, 2, 3, 4]},
                      date_parser=parse,
                      index_col='Datetime')

# Reindex to a common time step with forcing data
support = support.reindex(SNQ.time.values, fill_value=np.nan)

# -------------------------------------------------------------------------
# Convert to xarray Dataset
support = xr.Dataset(support)
support = support.rename({'Datetime':'time', 'Snowdepth': 'snowDepth'})

# -------------------------------------------------------------------------
# Surface temperature and snow presence
# Average together both surface IR thermometers to get a single time series
support['Tsrf'] = xr.concat([support.Tsrf_1, support.Tsrf_2], 'concatDim').mean(dim='concatDim')

# Take daily average, reindex to half-hourly time series, use in snow presence criteria
TsrfDaily = support.Tsrf.resample(how='mean', freq='d', dim='time', label='left')
TsrfDaily = TsrfDaily.reindex_like(support, method='ffill')

# bare ground when no snowdepth recorded or the daily surface temperature is above freezing
groundSurfTemp = support.Tsrf[(support.snowDepth == 0) | (TsrfDaily > 0.5)]

# snow covered ground when snow is observed and the surface temperature is below freezing
snowSurfTemp = support.Tsrf[(support.snowDepth > 0) & (TsrfDaily < 0.5)]

# Assign to support xarray.Dataset
support['groundTs'] = groundSurfTemp
support['snowTs'] = snowSurfTemp

# Create snow presence variable
snowPres = ((support.snowDepth > 0) | (TsrfDaily < 0.5))
support['SP'] = snowPres.astype(int)

//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1038: RuntimeWarning: invalid value encountered in greater
  if not reflexive
//anaconda/lib/python3.5/site-packages/xarray/core/variable.py:1038: RuntimeWarning: invalid value encountered in less
  if not reflexive


In [7]:
# -------------------------------------------------------------------------
# Add necessary support data to SNQ Dataset
# Many other variables are available, but I'm selecting only a small collection of them
SNQ['Albedo'] = support['Albedo']
SNQ['Tsrf'] = support['Tsrf']
SNQ['SP'] = support['SP']
SNQ['snowTs'] = support['snowTs']
SNQ['groundTs'] = support['groundTs']
SNQ['snowDepth'] = support['snowDepth']

# -------------------------------------------------------------------------
# Write netcdf
os.chdir(dirData)
SNQ.to_netcdf('SNQ.ModTsfc.nc')
print(SNQ)

<xarray.Dataset>
Dimensions:    (time: 45741)
Coordinates:
  * time       (time) datetime64[ns] 2012-10-01 2012-10-01T00:30:00 ...
Data variables:
    precip     (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWdwn      (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    LWdwn      (time) float64 286.9 286.9 286.5 286.5 286.5 286.5 286.5 ...
    Tair       (time) float64 5.856 5.856 5.875 6.061 6.267 6.214 5.994 ...
    WIND       (time) float64 0.068 0.126 0.06 0.126 0.045 0.099 0.199 0.116 ...
    Press      (time) float64 9.088e+04 9.086e+04 9.085e+04 9.085e+04 ...
    QS         (time) float64 0.005277 0.005341 0.005394 0.005419 0.005373 ...
    RH         (time) float64 83.29 84.28 85.0 84.3 82.4 83.71 84.1 84.68 ...
    Tdew       (time) float64 -267.3 -267.3 -267.3 -267.1 -266.9 -266.9 ...
    Albedo     (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
    Tsrf       (time) float64 nan nan nan nan nan nan nan nan nan nan nan ...
 